In [19]:
from functools import partial
import random

import numpy as np
import tables
import pandas as pd

In [2]:
fname = '../raw/total-3L.h5'

In [3]:
def unbound_range(start=0):
    current_value = start
    while True:
        yield current_value
        current_value += 1

In [4]:
#filtering order matters for performance...

def accessibility_filter(recs):
    for rec in recs:
        if rec['accessibility']:
            yield rec


def subsample_filter(recs, cut):
    for rec in recs:
        if random.random() < cut:
            yield rec


#start filter vs end filter
def start_filter(recs, start):
    #There would be more efficient ways of doing this...
    for rec in recs:
        pos = rec['pos']
        if pos < start:
            continue
        else:
            yield rec

            
def end_filter(recs, end):
    for rec in recs:
        pos = rec['pos']
        if pos > end:
            #we can do this because the input is ordered!
            raise StopIteration
        else:
            yield rec
            

def biallelic_filter(recs):
    for rec in recs:
        alt = rec['alt']
        if len([x for x in alt if x != b'']) == 1:
            yield rec

In [5]:
def get_all_data(store):
    genotype_array = store.get_node('/3L/calldata/genotype').iterrows()
    called_array = store.get_node('/3L/calldata/is_called').iterrows()
    accessibility_array = store.get_node('/3L/variants/Accessible').iterrows()
    pos_array = store.get_node('/3L/variants/POS').iterrows()
    alt_array = store.get_node('/3L/variants/ALT').iterrows()
    #this will not work on Python 2 (unless you use ittertools.zip)
    for genotype, accessibility, pos, alt, called in zip(
        genotype_array, accessibility_array, pos_array, alt_array, called_array):
        yield {
            'genotype': genotype,
            'called': called,
            'accessibility': accessibility,
            'pos': pos,
            'alt': alt,
        }


In [6]:
start_function = partial(start_filter, start=10000000)
end_function = partial(end_filter, end=30000000)

In [7]:
#python 3.5!
def filter_partial_subsample(recs, subsample_function=partial(subsample_filter, cut=0.1)):
    yield from biallelic_filter(end_function(start_function(accessibility_filter(subsample_function(recs)))))

def filter_partial_subsample_slow(recs, subsample_function=partial(subsample_filter, cut=0.1)):
    yield from end_function(start_function(accessibility_filter(subsample_function(biallelic_filter(recs)))))
    
def filter_partial_no_subsample(recs, subsample_function=partial(subsample_filter, cut=0.1)):
    yield from end_function(start_function(accessibility_filter(biallelic_filter(recs))))
    
def filter_no_subsample(recs):
    yield from accessibility_filter(biallelic_filter(recs))
    
def filter_subsample(recs, subsample_function=partial(subsample_filter, cut=0.1)):
    yield from accessibility_filter(biallelic_filter(subsample_function(recs)))

In [8]:
store = tables.open_file(fname, 'r')

In [9]:
#%time sum(1 for x in filter_partial_subsample(get_all_data(store)))

In [10]:
#%time print(sum(1 for x in filter_partial_subsample_slow(get_all_data(store))))

In [11]:
%time num_snps = sum(1 for x in filter_no_subsample(get_all_data(store)))
print(num_snps)

CPU times: user 6min 30s, sys: 2.47 s, total: 6min 32s
Wall time: 6min 33s
7449486


In [12]:
def create_hdf5(name, samples, recs, size):
    #size will mostbly be approxiamte because it is random
    w = tables.open_file(name, mode='w', filters=tables.Filters(complib='zlib', complevel=5))
    genotypes = tables.EArray(w.root, 'genotypes', tables.BoolAtom(),
                         expectedrows=size, shape=(0, len(samples), 2))
    positions = tables.EArray(w.root, 'positions', tables.IntAtom(),
                         expectedrows=size, shape=(0,))
    for rec in recs:
        old_genotypes = rec['genotype']
        #Only works for bi-allelic and no missing data
        min_val = old_genotypes.min()
        positions.append([rec['pos']])
        genotypes.append([np.array(list(map(lambda x: x == min_val, old_genotypes)))])
    w.create_array(w.root, 'samples', samples, 'Sample ids')    
    w.close()

In [13]:
samples = store.get_node('/3L/samples').read()

In [14]:
create_hdf5('subsample_010.h5', samples, filter_subsample(get_all_data(store)), int(num_snps * 0.1))

In [15]:
create_hdf5('subsample_001.h5', samples,
            filter_subsample(get_all_data(store),
                             subsample_function=partial(subsample_filter, cut=0.01)),
            int(num_snps * 0.01))

In [20]:
create_hdf5('subsample_000c1.h5', samples,
            filter_subsample(get_all_data(store),
                             subsample_function=partial(subsample_filter, cut=0.001)),
            int(num_snps * 0.001))

In [16]:
subsample_function = partial(subsample_filter, cut=0.01)
create_hdf5('partial_subsample_001.h5', samples,
            filter_partial_subsample(get_all_data(store), subsample_function=partial(subsample_filter, cut=0.01)),
            int(num_snps * 0.01 * 0.5))
#assuming circa half SNPs (20M of 49M but centromere and telomere outside)

In [17]:
create_hdf5('full.h5', samples, filter_no_subsample(get_all_data(store)), num_snps)

In [18]:
#meta = pd.DataFrame.from_csv('../raw/samples.all.txt', sep='\t')
#pandas_sub_store = pd.HDFStore('subsample.h5')
#pandas_sub_store